# Basic Agent Sudy

***Disclaimer***: This file referenced some files in other directories. In order to have working cross referencing it's recommended to start the notebook server from the root directory (`Grid2Op`) of the package and __not__ in the `getting_started` sub directory:
```bash
cd Grid2Op
jupyter notebook
```

***NB*** For more information about how to use the package, a general help can be built locally (provided that sphinx is installed on the machine) with:
```bash
cd Grid2Op
make html
```
from the top directory of the package (usually `Grid2Op`).

Once build, the help can be access from [here](../documentation/html/index.html)

It is recommended to have a look at the [0_basic_functionalities](0_basic_functionalities.ipynb), [1_Observation_Agents](1_Observation_Agents.ipynb), [2_Action_GridManipulation](3_TrainingAnAgent.ipynb) and [3_TrainingAnAgent](3_TrainingAnAgent.ipynb) notebooks before getting into this one.

**Objectives**

In this notebook we will expose how to study an Agent. For this notebook to be interested, we first use a dummy agent, and then we look at how to study his behaviour from the file saved.

It is more than recommended to know how to define an Agent and use a Runner before doing this tutotial!

## Evaluate the performance of a simple Agen

In [1]:
import os
import sys
import grid2op
import numpy as np

import seaborn as sns
import plotly.graph_objects as go

from grid2op.Agent import PowerLineSwitch
from grid2op.Reward import L2RPNReward
from grid2op.Runner import Runner
from grid2op.ChronicsHandler import GridStateFromFileWithForecasts, Multifolder
path_agent = "study_agent_getting_started"

In the next cell we evaluate the agent "PowerLineSwitch" and save the results of this evaluation in "study_agent_getting_started"

In [ ]:
scoring_function = L2RPNReward
# make a runner
runner = Runner(init_grid_path=grid2op.CASE_14_FILE, # this should be the same grid as the one the agent is trained one
                path_chron=grid2op.CHRONICS_MLUTIEPISODE,  # chronics can changed of course
                gridStateclass=Multifolder, # the class of chronics can changed too
                gridStateclass_kwargs={"gridvalueClass": GridStateFromFileWithForecasts},  # so this can changed too
                names_chronics_to_backend = grid2op.NAMES_CHRONICS_TO_BACKEND,  # this also can changed
                agentClass=PowerLineSwitch,
                rewardClass=scoring_function
                )
res = runner.run(path_save=path_agent, nb_episode=2)
print("The results for the evaluated agent are:")
for chron_name, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

## Looking at the results, understand the behaviour of the Agent

The content of the folder is the following:

In [ ]:
!ls $path_agent

As we notice, there are 2 json that represents the action space and the observation space, used to read the data store. These objects can be loaded this way:

In [ ]:
from grid2op.Utils import ActionSpace, ObservationSpace

action_space = ActionSpace.from_dict("study_agent_getting_started/dict_action_space.json")
env_modif_space = ActionSpace.from_dict("study_agent_getting_started/dict_env_modification_space.json")
observation_space = ObservationSpace.from_dict("study_agent_getting_started/dict_observation_space.json")

Now we can load the data corresponding to episode 1 for example, we can load the actions and the observations and de-serialize them properly into proper objects:

In [ ]:
path_episode_1 = os.path.join(path_agent, "1")
actions_npy = np.load(os.path.join(path_episode_1, "actions.npy"))
li_actions = []
for i in range(actions_npy.shape[0]):
    tmp = action_space.from_vect(actions_npy[i,:])
    li_actions.append(tmp)
    
observations_npy = np.load(os.path.join(path_episode_1, "observations.npy"))
li_observations = []
for i in range(observations_npy.shape[0]):
    tmp = observation_space.from_vect(observations_npy[i,:])
    li_observations.append(tmp)
    
env_modifications = np.load(os.path.join(path_episode_1, "env_modifications.npy"))
li_env_modifs = []
for i in range(env_modifications.shape[0]):
    tmp = env_modif_space.from_vect(env_modifications[i,:])
    li_env_modifs.append(tmp)

## Inspect the actions

And now we can start to study the given agent, for example, let's inspect its actions and wonder how many powerlines it has disconnected (for example, this is probably not the best thing to do here...)

In [ ]:
line_disc = 0
line_reco = 0
for act in li_actions:
    dict_ = act.as_dict() # representation of an action as a dictionnary, see the documentation for more information
    if "set_line_status" in dict_:
        line_reco +=  dict_["set_line_status"]["nb_connected"]
        line_disc +=  dict_["set_line_status"]["nb_disconnected"]
line_disc

We can also wonder how many times this Agent acted on the powerline with id 1, and inspect how many times it has change its status:

In [ ]:
line_disconnected = 0
for act in li_actions:
    dict_ = act.effect_on(line_id=1) # which effect has this action action on the substation with id 1
    # other objects are: load_id, gen_id, line_id or substation_id
    if dict_['set_line_status'] == 1 :
        line_disconnected += 1
line_disconnected

## Inspect the modification of the environment

For example, we might want to inspect the number of hazards and maintenance of a total scenario, to see how difficult it was.

In [ ]:
nb_hazards = 0
nb_maintenance = 0
for act in li_env_modifs:
    dict_ = act.as_dict() # representation of an action as a dictionnary, see the documentation for more information
    if "nb_hazards" in dict_:
        nb_hazards += 1
    if "nb_maintenance" in dict_:
        nb_maintenance += 1
nb_maintenance

In [ ]:
dict_

## Inspect the observations

For example, let's look at the value consumed by load 1. For this cell to work, it requires plotly for displaying the results.

In [ ]:
import plotly.graph_objects as go
load_id = 1
# extract the data
val_load1 = np.zeros(len(li_observations))
for i, obs in enumerate(li_observations):
    dict_ = obs.state_of(load_id=load_id) # which effect has this action action on the substation with id 1
    # other objects are: load_id, gen_id, line_id or substation_id
    # see the documentation for more information.
    val_load1[i] = dict_['p']

# plot it
fig = go.Figure(data=[go.Scatter(x=[i for i in range(len(val_load1))],
                                 y=val_load1)])
fig.update_layout(title="Consumption of load {}".format(load_id),
                 xaxis_title="Time step",
                 yaxis_title="Load (MW)")
fig.show()

Or the values of generator 3 (it's supposed to represent a solar energy source)

In [ ]:
gen_id = 3
# extract the data
val_lgen3 = np.zeros(len(li_observations))
for i, obs in enumerate(li_observations):
    dict_ = obs.state_of(gen_id=gen_id) # which effect has this action action on the substation with id 1
    # other objects are: load_id, gen_id, line_id or substation_id
    # see the documentation for more information.
    val_lgen3[i] = dict_['p']

# plot it
fig = go.Figure(data=[go.Scatter(x=[i for i in range(len(val_lgen3))],
                                 y=val_lgen3)])
fig.update_layout(title="Production of generator {}".format(gen_id),
                 xaxis_title="Time step",
                 yaxis_title="Production (MW)")
fig.show()

In the same fashion, we might want to get the flows on powerline connecting bus 3 to bus 4 (without knowing its id by using the appropriate method of the observation_space):

In [ ]:
from_ = 3
to_ = 4
found_ids = observation_space.get_lines_id(from_=from_, to_=to_)
line_id = found_ids[0]

# extract the data
val_l3_4 = np.zeros(len(li_observations))
for i, obs in enumerate(li_observations):
    dict_ = obs.state_of(line_id=line_id) # which effect has this action action on the substation with id 1
    # other objects are: load_id, gen_id, line_id or substation_id
    # see the documentation for more information.
    val_l3_4[i] = dict_["origin"]['a']

# plot it
fig = go.Figure(data=[go.Scatter(x=[i for i in range(len(val_l3_4))],
                                 y=val_l3_4)])
fig.update_layout(title="Production of generator {}".format(gen_id),
                 xaxis_title="Time step",
                 yaxis_title="Production (MW)")
fig.show()

## Quick display of a grid using an observation

Bellow you can find an example on how to plot a observation and the underlying powergrid. This is an example, the results doesn't look really great. It uses plotly and requires the layout of the grid (eg the coordinates of the substations) to be specified.

Note also that this code is not optimized at all.

In [ ]:
# Some utilities to plot substation, lines or get the color id for the colormap.
def draw_sub(pos, radius=50):
    pos_x, pos_y = pos
    res = go.layout.Shape(
        type="circle",
        xref="x",
        yref="y",
        x0=pos_x-radius,
        y0=pos_y-radius,
        x1=pos_x+radius,
        y1=pos_y+radius,
        line_color="LightSeaGreen"
    )
    return res

def get_col(rho):
    if rho < 0.7:
        return 0
    if rho < 0.8:
        return 1
    if rho < 0.9:
        return 2
    if rho < 1.:
        return 3
    if rho < 1.1:
        return 5
    return 6

def draw_line(pos_sub_or, pos_sub_ex, rho, color_palette):

    x_0, y_0 = pos_sub_or
    x_1, y_1 = pos_sub_ex
    
    res = go.layout.Shape(
            type="line",
            xref="x",
            yref="y",
            x0=x_0,
            y0=y_0,
            x1=x_1,
            y1=y_1,
            line=dict(
                color=color_palette[get_col(rho)]#'cymk{}'.format(color_palette(rho))#color_palette(rho)
            )
        )
    return res

In [ ]:
# define a color palette, whatever...
sns.set()
# pal = sns.dark_palette("palegreen")
pal = sns.color_palette("coolwarm", 7)
cols = pal.as_hex()

In [ ]:
# init the plot
graph_layout = [(280, -81), (100, -270), (-366, -270), (-366, -54), (64, -54), (64, 54), (-366, 0), 
                (-438, 0), (-326, 54), (-222, 108), (-79, 162), (152, 270), (64, 270), (-222, 216)]
fig = go.Figure()
radius_sub = 25
# draw substation
substation_layout = [draw_sub(el,radius=radius_sub) for i,el in enumerate(graph_layout)]

# draw name of substation
fig.add_trace(go.Scatter(x=[el for el,_ in graph_layout],
                         y=[el for _, el in graph_layout],
                         text=["sub_{}".format(i) for i,_ in enumerate(graph_layout)],
                         mode="text",
                        showlegend=False))
# get an observation
obs_plot = li_observations[231]

# draw powerline
lines = []
text = []
for line_id, rho in enumerate(obs_plot.rho):
    # the next 5 lines are always the same, for each observation, it makes sense to compute it once
    # and then reuse it
    state = obs_plot.state_of(line_id=line_id)
    sub_or_id = state["origin"]["sub_id"]
    sub_ex_id = state["extremity"]["sub_id"]
    pos_or = graph_layout[sub_or_id]
    pos_ex = graph_layout[sub_ex_id]
    # TODO here: make there the powerline are going to the right place, and not at the center of the circle
    # it is not done here so all lines intersect at the center of the grid!
    
    #NB to get on which bus a powerline is connected its:
    bus_id_or = state["origin"]["bus"]
    bus_id_ex = state["extremity"]["bus"]
    
    # this depends on the grid
    # on this powergrid, thermal limit are not set at all. They are basically random.
    # so i multiply them by 300
    rho *= 300
    lines.append(draw_line(pos_or, pos_ex,
                           rho=rho, color_palette=cols))
    # TODO adjust position of labels...
    fig.add_trace(go.Scatter(x=[(pos_or[0] + pos_ex[0])/2],
                         y=[(pos_or[1] + pos_ex[1])/2],
                         text=["{:.1f}%".format(rho*100)],
                         mode="text",
                            showlegend=False))
    
fig.update_layout(shapes=substation_layout+lines)

# update legend, background color etc.
fig.update_xaxes(range=[np.min([el for el,_ in graph_layout])-2*radius_sub,
                        np.max([el for el,_ in graph_layout])+2*radius_sub], zeroline=False)
fig.update_yaxes(range=[np.min([el for _, el in graph_layout])-2*radius_sub,
                        np.max([el for _, el in graph_layout])+2*radius_sub])
fig.update_layout(
    margin=dict(
        l=20,
        r=20,
        b=100
    ),
    height=600,
    width=800,
    plot_bgcolor="white"
)
fig.show()

## Synching Observation and Action

As stated in the documentation, at row i, it's the observation at time "i" and the action at time "i". So at row i of the numpy matrices, we see what the agent saw when he took his actions. We have "an agent view".

In case we want to see the impact of an Action, it is then necessary to:

- look at action i
- look at observation i+1